**Download the datasets**

In [11]:
import yfinance as yf # 資料來源： Yahoo Finance
import numpy as np

# Define USD-based currency pairs to download (Yahoo Finance ticker format)
symbols = ['USDJPY=X', 'EURUSD=X', 'GBPUSD=X']

# Download historical FX data
fx_data = {}
for symbol in symbols:
    data = yf.download(symbol, period="12y", interval="1d")
    fx_data[symbol] = data['Close']
    print(f"{symbol} - {len(data)} rows downloaded.")

# Convert to pure numpy arrays
fx_numpy_list = [fx_data[s].to_numpy() for s in symbols]

# Align all currency pairs to the same length
min_len = min(len(arr) for arr in fx_numpy_list)
fx_numpy_list = [arr[-min_len:] for arr in fx_numpy_list]

# Combine into shape (3, N) → each currency pair is one row (dimension)
fx_combined = np.stack(fx_numpy_list, axis=0)
fx_data = fx_combined.reshape(3, -1)

C:\Users\HP\AppData\Local\Temp\ipykernel_28436\3891507856.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, period="12y", interval="1d")
[*********************100%***********************]  1 of 1 completed
C:\Users\HP\AppData\Local\Temp\ipykernel_28436\3891507856.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, period="12y", interval="1d")


USDJPY=X - 3124 rows downloaded.


[*********************100%***********************]  1 of 1 completed
C:\Users\HP\AppData\Local\Temp\ipykernel_28436\3891507856.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, period="12y", interval="1d")


EURUSD=X - 3124 rows downloaded.


[*********************100%***********************]  1 of 1 completed

GBPUSD=X - 3124 rows downloaded.


In [12]:
import pandas as pd

# Transpose → shape becomes (N, 3), suitable for DataFrame format
fx_data = fx_data.T  # Example: shape (N, 3), N = number of days

# Define currency pair column names (without "=X" for cleaner column names)
symbols = ['USDJPY', 'EURUSD', 'GBPUSD']

# Convert to DataFrame
df = pd.DataFrame(fx_data, columns=symbols)

# Save to Excel
df.to_excel('fx_data.xlsx', index=False)

print("Saved to fx_data.xlsx")

Saved to fx_data.xlsx


**Train Process**

***Step 1：切訓練集 + 測試集（預測 horizon = 1***

In [13]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

window_size = 30  # 定義序列長度

# Step 1: 檢查資料長度
num_days = fx_data.shape[0]
print(f"共有 {num_days} 天的資料")

# Step 2: 切分比例
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

train_days = int(num_days * train_ratio)
val_days = int(num_days * val_ratio)

print(f"訓練集長度: {train_days} 天")
print(f"驗證集長度: {val_days} 天")
print(f"測試集長度: {num_days - train_days - val_days} 天")

# Step 3: 正規化資料
scaler = MinMaxScaler()
fx_scaled = scaler.fit_transform(fx_data)  # shape: (N, 3)

# Step 4: 建立序列資料 (用 window_size)
def create_sequences(data, start_idx, end_idx, window_size):
    X, y = [], []
    for i in range(start_idx, end_idx - window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size])
    return np.array(X), np.array(y)

# Step 5: 切分序列
X_train, y_train = create_sequences(fx_scaled, 0, train_days, window_size)
X_val, y_val = create_sequences(fx_scaled, train_days, train_days + val_days, window_size)
X_test, y_test = create_sequences(fx_scaled, train_days + val_days, num_days, window_size)


共有 3124 天的資料
訓練集長度: 2186 天
驗證集長度: 468 天
測試集長度: 470 天


*** Step 2：建立並訓練模型（用 LSTM 為例***

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 建立 GRU 模型
model = Sequential()
model.add(GRU(64, input_shape=(window_size, 3)))
model.add(Dense(3))  # 三個幣種輸出
model.compile(optimizer='adam', loss='mse')

# 加入 callbacks（避免過擬合）
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint("best_gru_model.h5", save_best_only=True)
]

# 訓練
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    verbose=1,
    callbacks=callbacks,
    shuffle=False
)

# 儲存最終模型（可選）
model.save("fx_model_gru.h5")
print("模型已儲存為 fx_model_gru.h5")


Epoch 1/30


c:\Users\HP\anaconda3\envs\mll\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


64/68 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2481

68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.2367 - val_loss: 0.0604
Epoch 2/30
66/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0252

68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0248 - val_loss: 0.0427
Epoch 3/30
65/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0147

...

68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 2.4773e-04 - val_loss: 8.8537e-04
Epoch 28/30
63/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.4597e-04

68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 2.4495e-04 - val_loss: 8.0749e-04
Epoch 29/30
63/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.4423e-04

68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 2.4314e-04 - val_loss: 7.4602e-04
Epoch 30/30
61/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.4464e-04

68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 2.4332e-04 - val_loss: 7.0385e-04


模型已儲存為 fx_model_gru.h5


***Step 3：開始做 Rolling 預測 + 驗證***

In [19]:
# 預測測試集
preds = model.predict(X_test)  # shape: (N_test, 3)
true_vals = y_test             # shape: (N_test, 3)

# 還原為實際價格（逆轉正規化）
preds_real = scaler.inverse_transform(preds)
true_vals_real = scaler.inverse_transform(true_vals)

# 可選：轉為 list 存起來（若後續畫圖用）
preds = preds_real.tolist()
true_vals = true_vals_real.tolist()


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


***Step 4：計算誤差指標（RMSE、MAE)***

In [20]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

preds = np.array(preds)
true_vals = np.array(true_vals)

rmse = np.sqrt(mean_squared_error(true_vals, preds))
mae = mean_absolute_error(true_vals, preds)

print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")


RMSE: 0.8236
MAE: 0.4124


In [21]:
import matplotlib.pyplot as plt
import os

currency_names = ['USDJPY', 'EURUSD', 'GBPUSD']

# 建立 results 資料夾（若尚未存在）
save_dir = 'results'
os.makedirs(save_dir, exist_ok=True)

def get_unique_filename(base_name, ext='png'):
    """回傳不重複的檔案名稱（在 results 資料夾內）"""
    counter = 1
    filename = os.path.join(save_dir, f"{base_name}.{ext}")
    while os.path.exists(filename):
        filename = os.path.join(save_dir, f"{base_name}_{counter}.{ext}")
        counter += 1
    return filename

for i in range(3):
    plt.figure(figsize=(20, 4))
    plt.plot([t[i] for t in true_vals], label='True')
    plt.plot([p[i] for p in preds], label='Predicted')
    plt.title(f"Rolling Prediction - {currency_names[i]}")
    plt.xlabel("Days")
    plt.ylabel("Exchange Rate")
    plt.legend()
    plt.grid(True)

    filename = get_unique_filename(f"rolling_prediction_{currency_names[i]}")
    plt.savefig(filename)
    plt.close()
    print(f"Saved: {filename}")


Saved: results\rolling_prediction_USDJPY_24.png
Saved: results\rolling_prediction_EURUSD_24.png
Saved: results\rolling_prediction_GBPUSD_24.png
